In [1]:
import numpy as np
import pickle
import os
import signatory

seed = 2023

ImportError: dlopen(/Users/rchan/opt/miniconda3/envs/nlpsig-networks/lib/python3.8/site-packages/signatory/_impl.cpython-38-darwin.so, 0x0002): symbol not found in flat namespace '___kmpc_barrier'

In [4]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

In [5]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [6]:
%run ../load_anno_mi.py

In [7]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-06-27 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-27 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-27 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-27 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-27 00:00:34


In [8]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [9]:
dim_reduce_methods = ["gaussian_random_projection", "umap"]

## Using log signature

In [22]:
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3), (6, 4), (3, 7)]
[signatory.logsignature_channels(channels, depth)
 for (channels, depth) in log_signature_dimensions_and_sig_depths]

[406, 385, 406, 508]

## `history_length=20`

### Using UMAP

In [ ]:
history_length = 50
ffn_logsignature_umap_kfold_20, best_ffn_logsignature_umap_kfold_20, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    window_sizes=[history_length],
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_{history_length}_umap_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
ffn_logsignature_umap_kfold_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
ffn_logsignature_umap_kfold_20["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_umap_kfold_20["f1_scores"]).mean(axis=0)

### Using random projections

In [ ]:
history_length = 20
ffn_logsignature_umap_kfold_20, best_ffn_logsignature_grp_kfold_20, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    window_sizes=[history_length],
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_{history_length}_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
ffn_logsignature_umap_kfold_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_ffn_logsignature_grp_kfold_20["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_grp_kfold_20["f1_scores"]).mean(axis=0)

## `history_length=50`

### Using UMAP

In [ ]:
history_length = 50
ffn_logsignature_umap_kfold_20, best_ffn_logsignature_umap_kfold_20, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    window_sizes=[history_length],
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_{history_length}_umap_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
ffn_logsignature_umap_kfold_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
ffn_logsignature_umap_kfold_20["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_umap_kfold_20["f1_scores"]).mean(axis=0)

### Using random projections

In [ ]:
history_length = 50
ffn_logsignature_umap_kfold_50, best_ffn_logsignature_grp_kfold_50, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    window_sizes=[history_length],
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_{history_length}_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
ffn_logsignature_umap_kfold_50.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_ffn_logsignature_grp_kfold_50["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_umap_kfold_50["f1_scores"]).mean(axis=0)